In [ ]:
import polars as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import glob
import os
import json
from tqdm import tqdm

# ------------------------------------------------------------------
# 1. 파일 필터링 및 데이터 로드 (202412 ~ 202511)
# ------------------------------------------------------------------
base_path = "/content/drive/MyDrive/3팀/데이터/parquet"
files_202412 = glob.glob(os.path.join(base_path, "202412.parquet"))
files_2025 = glob.glob(os.path.join(base_path, "2025*.parquet"))

all_files = files_202412 + files_2025

print(f"📦 읽어올 파일 개수: {len(all_files)}개")

# 데이터 스캔 및 1000만 건 샘플링 (admi_cty_no 제거)
raw_df = (
    pl.scan_parquet(all_files)
    .select(["card_tpbuz_nm_2", "hour", "sex", "age", "day", "cnt"]) # area 제거
    .collect(engine="streaming")
)

sampling_fraction = 10000000 / len(raw_df)
train_df_raw = raw_df.sample(fraction=sampling_fraction, seed=42)
print(f"✅ 샘플링 완료: {len(train_df_raw):,} 건 (전체의 {sampling_fraction*100:.1f}%)")

# ------------------------------------------------------------------
# 2. 매핑 및 데이터 타입 최적화 (메모리 절약)
# ------------------------------------------------------------------
def get_mapping(df, col_name):
    unique_vals = df[col_name].cast(pl.String).unique().to_list()
    mapping = {val: i for i, val in enumerate(unique_vals)}
    return mapping, len(unique_vals)

biz_map, n_classes = get_mapping(train_df_raw, "card_tpbuz_nm_2")
sex_map, n_sex = get_mapping(train_df_raw, "sex")
inv_biz_map = {v: k for k, v in biz_map.items()}

# 데이터 타입 경량화 및 인덱스 변환
train_df = train_df_raw.with_columns([
    (pl.col("cnt").cast(pl.Float32) + 1).log().alias("log_cnt"),
    pl.col("card_tpbuz_nm_2").cast(pl.String).replace(biz_map).cast(pl.Int32),
    pl.col("sex").cast(pl.String).replace(sex_map).cast(pl.Int8),
    (pl.col("age").cast(pl.Int8) - 2).alias("age_idx"),
    pl.col("day").cast(pl.Int8),
    pl.col("hour").cast(pl.Float32)
])

# ------------------------------------------------------------------
# 3. 모델 및 데이터셋 정의
# ------------------------------------------------------------------
class FoodRecommender(nn.Module):
    def __init__(self, n_classes, n_sex):
        super(FoodRecommender, self).__init__()
        # area_emb 제거
        self.sex_emb = nn.Embedding(n_sex, 2)
        self.age_emb = nn.Embedding(5, 4)
        self.day_emb = nn.Embedding(8, 4)
        # 입력 차원: 2(sex) + 4(age) + 4(day) + 1(hour) = 11
        self.fc = nn.Sequential(
            nn.Linear(11, 128), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(128, 64), nn.ReLU(),
            nn.Linear(64, n_classes)
        )

    def forward(self, hour, sex, age, day):
        x = torch.cat([
            self.sex_emb(sex), self.age_emb(age),
            self.day_emb(day), hour.unsqueeze(1)
        ], dim=1)
        return self.fc(x)

class FoodDataset(Dataset):
    def __init__(self, df):
        self.hour = df["hour"].to_numpy()
        self.sex = df["sex"].to_numpy()
        self.age = df["age_idx"].to_numpy()
        self.day = df["day"].to_numpy()
        self.label = df["card_tpbuz_nm_2"].to_numpy()
        self.weights = df["log_cnt"].to_numpy()

    def __len__(self): return len(self.label)
    def __getitem__(self, idx):
        return (torch.tensor(self.hour[idx], dtype=torch.float32),
                torch.tensor(self.sex[idx], dtype=torch.long),
                torch.tensor(self.age[idx], dtype=torch.long),
                torch.tensor(self.day[idx], dtype=torch.long),
                torch.tensor(self.label[idx], dtype=torch.long),
                torch.tensor(self.weights[idx], dtype=torch.float32))

# ------------------------------------------------------------------
# 4. 학습 실행 (1,000만 건 & 코랩 고성능 GPU 최적화)
# ------------------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FoodRecommender(n_classes, n_sex).to(device)

# 배치 사이즈를 65536으로 키우고, pin_memory를 True로 두어 GPU 전송 속도를 높입니다.
loader = DataLoader(
    FoodDataset(train_df), 
    batch_size=65536,
    shuffle=True, 
    num_workers=4,     # 코랩 사양에 따라 2~4 설정
    pin_memory=True
)

# 제거하고 돌려야 했지만 실수로 넣고 실행해서 5 epoch 2번 실행으로 10회 학습된 모델 생성

# # 배치 사이즈가 커진 만큼 LR을 아주 살짝만 높여봅니다.
# optimizer = torch.optim.Adam(model.parameters(), lr=0.0015) 
# criterion = nn.CrossEntropyLoss(reduction='none')

# print("🚀 학습 시작...")
# model.train()
# for epoch in range(5):
#     loss_sum = 0
#     for hour, sex, age, day, label, weight in tqdm(loader, desc=f"Epoch {epoch+1}"):
#         hour, sex, age, day, label, weight = \
#             hour.to(device), sex.to(device), age.to(device), \
#             day.to(device), label.to(device), weight.to(device)

#         optimizer.zero_grad()
#         output = model(hour, sex, age, day)
#         loss = (criterion(output, label) * weight).mean()
#         loss.backward()
#         optimizer.step()
#         loss_sum += loss.item()
#     print(f"✅ Epoch {epoch+1} Loss: {loss_sum/len(loader):.4f}")

# [변경 포인트] 저장 경로를 구글 드라이브로 설정
save_path = "/content/drive/MyDrive/3팀/출력" # 본인의 드라이브 경로로 수정
os.makedirs(save_path, exist_ok=True)

print("🚀 학습 시작 (1,000만 건)...")
model.train()
for epoch in range(5):
    loss_sum = 0
    for hour, sex, age, day, label, weight in tqdm(loader, desc=f"Epoch {epoch+1}"):
        hour, sex, age, day, label, weight = \
            hour.to(device), sex.to(device), age.to(device), \
            day.to(device), label.to(device), weight.to(device)

        optimizer.zero_grad()
        output = model(hour, sex, age, day)
        loss = (criterion(output, label) * weight).mean()
        loss.backward()
        optimizer.step()
        loss_sum += loss.item()
    
    # [안전 장치] 매 에폭이 끝날 때마다 임시 가중치 저장 (끊김 대비)
    torch.save(model.state_dict(), os.path.join(save_path, f"model_epoch_{epoch+1}.pth"))
    print(f"✅ Epoch {epoch+1} 완료 및 임시 저장됨. Loss: {loss_sum/len(loader):.4f}")

# ------------------------------------------------------------------
# 6. 최종 모델 및 매핑 정보 저장 (필수!)
# ------------------------------------------------------------------
# 1) 모델 가중치 저장
torch.save(model.state_dict(), os.path.join(save_path, "food_recommender_v1_10m.pth"))

# 2) 매핑 정보 저장 (이게 있어야 나중에 불러올 수 있음)
mapping_data = {
    "biz_map": biz_map,
    "sex_map": sex_map,
    "inv_biz_map": inv_biz_map,
    "n_classes": n_classes,
    "n_sex": n_sex
}
with open(os.path.join(save_path, "category_mappings.json"), "w", encoding="utf-8") as f:
    json.dump(mapping_data, f, ensure_ascii=False, indent=4)

print(f"✨ 모든 파일이 {save_path}에 안전하게 저장되었습니다.")

In [ ]:
# 7. [핵심] 10회 학습된 모델로 최종 JSON 결과 생성
print("📂 10회 학습 모델 기반 최종 JSON 생성 중...")
def get_hour_idx(h):
    if 0 <= h < 7: return 0
    elif 7 <= h < 9: return 1
    elif 9 <= h < 11: return 2
    elif 11 <= h < 13: return 3
    elif 13 <= h < 15: return 4
    elif 15 <= h < 17: return 5
    elif 17 <= h < 19: return 6
    elif 19 <= h < 21: return 7
    elif 21 <= h < 23: return 8
    elif 23 <= h < 24: return 9
    else: return 10

recommendation_map = {}
model.eval()
with torch.no_grad():
    for age in [2, 3, 4, 5, 6]:
        for sex in ['M', 'F']:
            for day in range(1, 8):
                for hour in range(24):
                    s_idx, ag_idx, d_idx, h_idx = sex_map[sex], age - 2, day - 1, get_hour_idx(hour)
                    sex_t, age_t, day_t, hour_t = torch.tensor([s_idx], device=device), torch.tensor([ag_idx], device=device), torch.tensor([d_idx], device=device), torch.tensor([float(h_idx)], device=device)
                    outputs = model(hour_t, sex_t, age_t, day_t)
                    probs = F.softmax(outputs, dim=1)
                    all_probs, all_indices = torch.topk(probs, n_classes)
                    ranked_list = [{"rank": i + 1, "category": inv_biz_map.get(all_indices[0][i].item(), "Unknown"), "probability": f"{round(all_probs[0][i].item() * 100, 2)}%"} for i in range(n_classes)]
                    recommendation_map[f"{age}_{sex}_{day}_{hour}"] = ranked_list

with open(os.path.join(save_path, "final_recommendation_10m.json"), "w", encoding="utf-8") as f:
    json.dump(recommendation_map, f, ensure_ascii=False, indent=4)

print(f"✨ 모든 작업 완료! {save_path}에서 10회 학습 결과물을 확인하세요.")

### 위 코드로 생성한 모델 직접 실행하는 방법(파이토치)

In [3]:
import torch
import torch.nn as nn
import json

# 1. 모델 구조 정의 (학습 때와 완벽히 동일해야 합니다)
class FoodRecommender(nn.Module):
    def __init__(self, n_classes, n_sex):
        super(FoodRecommender, self).__init__()
        self.sex_emb = nn.Embedding(n_sex, 2)
        self.age_emb = nn.Embedding(5, 4)
        self.day_emb = nn.Embedding(8, 4)
        self.fc = nn.Sequential(
            nn.Linear(11, 128), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(128, 64), nn.ReLU(),
            nn.Linear(64, n_classes)
        )

    def forward(self, hour, sex, age, day):
        x = torch.cat([
            self.sex_emb(sex),
            self.age_emb(age), 
            self.day_emb(day),
            hour.unsqueeze(1)
        ], dim=1)
        return self.fc(x)

# 2. 파일 로드 및 환경 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 저장된 매핑 정보 불러오기
with open("../output/category_mappings_10m.json", "r", encoding="utf-8") as f:
    mapping_data = json.load(f)

biz_map = mapping_data["biz_map"]
sex_map = mapping_data["sex_map"]
inv_biz_map = {int(k): v for k, v in mapping_data["inv_biz_map"].items()}
n_classes = mapping_data["n_classes"]
n_sex = mapping_data["n_sex"]

# 3. 모델 복원
model = FoodRecommender(n_classes, n_sex).to(device)
model.load_state_dict(torch.load("../output/food_recommender_v1_10m.pth", map_location=device))
model.eval() # 매우 중요: 추론 모드 설정

print(f"✅ 모델 복원 완료! (장치: {device})")

# 4. 추천 함수 정의
def predict_food(age, sex, day, hour):
    """
    age: 2, 3, 4, 5, 6 (각 20대~60대)
    sex: 'M' 또는 'F'
    day: 1~7 (월~일)
    hour: 0~23 (시)
    """
    def get_hour_idx(h):
        if 0 <= h < 7: return 0
        elif 7 <= h < 9: return 1
        elif 9 <= h < 11: return 2
        elif 11 <= h < 13: return 3
        elif 13 <= h < 15: return 4
        elif 15 <= h < 17: return 5
        elif 17 <= h < 19: return 6
        elif 19 <= h < 21: return 7
        elif 21 <= h < 23: return 8
        elif 23 <= h < 24: return 9
        else: return 10

    # 인덱스 변환
    s_idx = sex_map[sex]
    ag_idx = age - 2
    d_idx = day - 1
    h_idx = get_hour_idx(hour)
    
    # 텐서화
    sex_t = torch.tensor([s_idx], device=device)
    age_t = torch.tensor([ag_idx], device=device)
    day_t = torch.tensor([d_idx], device=device)
    hour_t = torch.tensor([float(h_idx)], device=device)
    
    with torch.no_grad():
        output = model(hour_t, sex_t, age_t, day_t)
        prob = torch.softmax(output, dim=1)
        values, indices = torch.topk(prob, 5) # 상위 5개 추출
        
        print(f"\n💡 [{age}0대 {sex}] 요일:{day}, 시간:{hour}시 추천 순위:")
        for i in range(5):
            category = inv_biz_map[indices[0][i].item()]
            percentage = values[0][i].item() * 100
            print(f"   {i+1}순위: {category} ({percentage:.2f}%)")

# 테스트 실행
predict_food(age=3, sex='F', day=6, hour=19) # 30대 여성 토요일 저녁 7시

✅ 모델 복원 완료! (장치: cpu)

💡 [30대 F] 요일:6, 시간:19시 추천 순위:
   1순위: 디저트/음료 (23.17%)
   2순위: 한식 (18.15%)
   3순위: 분식 (11.71%)
   4순위: 패스트푸드 (11.23%)
   5순위: 고기요리 (9.94%)


### 파이토치 설치가 어렵다면 json 결과물로 읽어오는 방법

In [5]:
import json

class FoodLookupService:
    def __init__(self, json_path):
        print("📂 추천 데이터를 로드 중입니다...")
        with open(json_path, 'r', encoding='utf-8') as f:
            self.data = json.load(f)
        print("✅ 로드 완료!")

    def get_recommendation(self, age, sex, day, hour):
        """
        age: 2, 3, 4, 5, 6 (각 20대~60대)
        sex: 'M' 또는 'F'
        day: 1~7 (월~일)
        hour: 0~23 (시)
        """
        # JSON 저장 시 사용했던 키 포맷 생성
        query_key = f"{age}_{sex}_{day}_{hour}"
        
        # 데이터 찾기
        result = self.data.get(query_key)
        
        if result:
            print(f"\n✨ [{age}0대 {'남성' if sex=='M' else '여성'}] {day}요일 {hour}시 추천 결과")
            # 상위 5개만 출력
            for item in result[:5]:
                print(f"   {item['rank']}위: {item['category']} (확률: {item['probability']})")
        else:
            print("❌ 해당 조건의 데이터를 찾을 수 없습니다. 입력값을 확인하세요.")

# 1. 서비스 초기화 (파일명이 다르면 수정하세요)
# 다른 컴퓨터라면 파일 경로를 정확히 입력해 주세요.
service = FoodLookupService("../output/final_recommendation_10m.json")

# 2. 사용 예시
# 예: 30대 여성, 토요일(6), 저녁 7시(19) 추천 메뉴는?
service.get_recommendation(age=3, sex='F', day=6, hour=19)

# 예: 50대 남성, 월요일(1), 아침 8시(8) 추천 메뉴는?
service.get_recommendation(age=5, sex='M', day=1, hour=8)

📂 추천 데이터를 로드 중입니다...
✅ 로드 완료!

✨ [30대 여성] 6요일 19시 추천 결과
   1위: 디저트/음료 (확률: 23.17%)
   2위: 한식 (확률: 18.15%)
   3위: 분식 (확률: 11.71%)
   4위: 패스트푸드 (확률: 11.23%)
   5위: 고기요리 (확률: 9.94%)

✨ [50대 남성] 1요일 8시 추천 결과
   1위: 디저트/음료 (확률: 36.04%)
   2위: 한식 (확률: 26.91%)
   3위: 분식 (확률: 14.75%)
   4위: 패스트푸드 (확률: 6.95%)
   5위: 별식/퓨전요리 (확률: 6.4%)
